In [ ]:
import os, sys, glob
from pathlib import Path

# adds the package path to the Python path to make sure all the local imports work fine 
if os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))) not in sys.path:
    sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))

# local imports 
from wp4.constants import POLLUTANTS, DATA_DIR_CAMS_AN, DATA_DIR_GFAS, DATA_DIR_PLOTS, DB_HOST, DB_NAME, DB_USER, DB_PASS
from wp4.baseline.spatial import get_spatial_baseline
from wp4.baseline.temporal import get_temporal_baseline

# import remaining packages needed for the script
import xarray as xr
import pandas as pd
import psycopg2 
from datetime import datetime, timedelta

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [ ]:
conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASS, host=DB_HOST)
cur = conn.cursor()

query = """
    SELECT id, datetime, ST_X(geometry), ST_Y(geometry), source, location, reference, type, info, frp
    FROM public.all_fire_events
    WHERE "frp" > 10 AND "frp" IS NOT NULL AND reference = 'Aqua' OR reference = 'Terra'
"""

df_fire_events = pd.read_sql_query(query,con=conn).rename(columns = {'st_x':'longitude', 'st_y':'latitude'})

conn.close()

In [ ]:
ds_flux = xr.open_dataset(Path(DATA_DIR_GFAS).joinpath('wildfire_flux.nc'))
ds_frp = xr.open_dataset(Path(DATA_DIR_GFAS).joinpath('frp.nc'))

In [ ]:
for ind, fe in df_fire_events.tail(30).iterrows():
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    fig.update_xaxes(title_text="Date")
    
    # Update yaxis properties
    fig.update_yaxes(title_text=f"Concentration µg m<sup>-2</sup> s<sup>-1</sup>")
    fig.update_yaxes(title_text=f"FRP", secondary_y=True)
    
    fig.update_layout(
        template=pio.templates["seaborn"],
        autosize=False,
        width=1000,
        height=500,
    )  
    
    for var in ds_flux.data_vars:

        ds_var = ds_flux[var].sel(time=slice(
                fe['datetime'].replace(minute=0) - timedelta(days=5),
                fe['datetime'].replace(minute=0) + timedelta(days=5)
            ))
        
        ds_var = ds_var * 1e+9
        
        ds_var = ds_var.sel(longitude=fe['longitude'], latitude=fe['latitude'], method='nearest')
        
        df_var = ds_var.to_dataframe()
        
        fig.add_trace(go.Scatter(  # add the baseline values
                x=df_var.index,
                y=df_var[var],
                mode='lines', 
                name=f'{var}'
        ))
        
    ds_frp_fe = ds_frp['frpfire'].sel(time=slice(
                fe['datetime'].replace(minute=0) - timedelta(days=5),
                fe['datetime'].replace(minute=0) + timedelta(days=5)
            ))
    ds_frp_fe = ds_frp_fe.sel(longitude=fe['longitude'], latitude=fe['latitude'], method='nearest')
        
    df_frp_fe = ds_frp_fe.to_dataframe()
    
    df_frp_fe = df_frp_fe['frpfire'].groupby('time').mean()
    
    fig.update_yaxes(rangemode="nonnegative")
    fig.update_yaxes(rangemode="nonnegative", secondary_y=True,)
        
    fig.add_trace(go.Scatter(  # add the CAMS analysis concentration data closest to the fireevent
                x=df_frp_fe.index,
                y=df_frp_fe.values,
                mode='lines',
                name='FRP',
                line={'color': 'black'}),
                secondary_y=True,
            )
        
    fig.show()